# Running the fleet of Virtual Wind Turbines and Edge Devices

**SageMaker Studio Kernel**: Data Science

After deploying the inference app on the 5 IoT Core devices(EC2 instances, in this case), its time to set up the virtual fleet of turbines and the simulator app along with it. In this exercise you will run a local application written in Python3 that simulates 5 Wind Turbines. This application will interact with IoT Core devices via IoT MQTT to exchange information.

Here you'll be the **Wind Turbine Farm Operator**. It's possible to visualize the data flowing from the sensors to the ML Model and analyze the anomalies. Also, you'll be able to inject noise (pressing some buttons) in the data to simulate potential anomalies with the equipment.

<table border="0" cellpading="0">
    <tr>
        <td align="center"><b>STEP-BY-STEP</b></td>
        <td align="center"><b>APPLICATION ARCHITECTURE</b></td>
    </tr>
    <tr>
        <td><img src="./../imgs/EdgeManagerWorkshop_Macro.png" width="500px"></img></td>
        <td><img src="./../imgs/ggv2_lab4_app_arch.png"  width="500px"></img></td>
    </tr>
</table>  

The components of the Simulator are:
 - [Simulator](simulator/simulator.py): Program that launches the virtual wind turbines and the edge devices. It also maintains a simple dashboard to show the data which is being sent for anomaly detection to the detector.
 - [Turbine](simulator/turbine.py): Virtual Wind Turbine. It reads the raw data collected from the 3D Prited Mini Turbine and stream it as a circular buffer. It also has a graphical representation in **IPython Widgets** that is rendered by the Simulator/Dashboard. It contains all the business logic of maintaining the virtual turbine.
 - [MQTT Client](simulator/mqttclient.py): An abstraction layer for MQTT communication providing convenience methods to publish and subscribe to topics.

First, let us restore variables stored in the previous exercise

### Install some dependencies

In [ ]:
!pip install ipywidgets
!pip install awsiotsdk

### Download the sensors data (streamed by the virtual wind turbines)

The below cell checks if the data is already downloaded.

if not, then it downloads the data. Otherwise it skips the downloading.

In [ ]:
import boto3
from importlib import reload
import logging
import os
import sys

bucket_name = ""

In [ ]:
sm_client = boto3.client('sagemaker')

In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

# Simulating The Wind Turbine Farm
Now its time to run our simulator and start playing with the turbines and with the anomalies
 > After clicking on **Start**, each turbine will start buffering some data. It takes a few seconds but after completing this process, the application runs in real-time   
 > Try to press some buttons while the simulation is running, to inject noise in the data and see some anomalies  


In [ ]:
sys.path.insert(0, './../fleet_simulator')

In [ ]:
import mqttclient
import simulator
import turbine

In [ ]:
reload(mqttclient)
reload(turbine)
reload(simulator)

simulator = simulator.WindTurbineFarmSimulator(2)

In [ ]:
simulator.start()
simulator.show()

Once your experiment is finished, stop the simulator. This will in turn also stops/halts all the turbines.

In [ ]:
simulator.halt()

 > If you want to experiment with the deployment process, with the wind farm running, go back to [Exercise #4](./../../04-Package-Deploy/04-package-using-ggv2.ipynb#Building-the-Deployment-Package-SageMaker-Edge-Manager) (in the last cells), replace the variable **model_version** by the constant (string) '2.0' in the Json document used by the IoT Job. Then, create a new IoT Job to simulate how to deploy new versions of the model. Go back to this exercise to see the results.

## Cleanup
Run the next cell only if you already finished exploring/hacking the content of the workshop.  
This code will delete all the resouces created so far

In [ ]:
import boto3
from botocore.exceptions import ClientError
import time
from shutil import rmtree

In [ ]:
iot_client = boto3.client('iot')
sm_client = boto3.client('sagemaker')
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
ggv2_client = boto3.client('greengrassv2')

In [ ]:
device_fleet_suffix = "fleet-1"
thing_group_name = "wind-ec2-fleet"

policy_name='WindTurbineFarmPolicy-{}'.format(device_fleet_suffix)
fleet_name='wind-turbine-farm-{}'.format(device_fleet_suffix)

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

In [ ]:
ggv2_client.list_deployments(
    targetArn="arn:aws:iot:{}:{}:thinggroup/{}".format(
        region,
        account_id,
        thing_group_name
    )
)

In [ ]:
deployment_id = ""

In [ ]:
#Cancel the deployment of greengrass v2
ggv2_client.cancel_deployment(deploymentId=deployment_id)

### Get Model Package Version for model component version

Since model component version is aligned with Model Registries version, we are retrieving those information

In [ ]:
model_package_group_arn = "mlops-iot-package-group"

In [ ]:
def describe_model_package(model_package_arn):
    try:
        model_package = sm_client.describe_model_package(
            ModelPackageName=model_package_arn
        )

        LOGGER.info("{}".format(model_package))

        if len(model_package) == 0:
            error_message = ("No ModelPackage found for: {}".format(model_package_arn))
            LOGGER.error("{}".format(error_message))

            raise Exception(error_message)

        return model_package
    except ClientError as e:
        stacktrace = traceback.format_exc()
        error_message = e.response["Error"]["Message"]
        LOGGER.error("{}".format(stacktrace))

        raise Exception(error_message)

In [ ]:
def get_approved_package(model_package_group_name):
    """Gets the latest approved model package for a model package group.

    Args:
        model_package_group_name: The model package group name.

    Returns:
        The SageMaker Model Package ARN.
    """
    try:
        # Get the latest approved model package
        response = sm_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            SortOrder="Descending",
            MaxResults=1,
        )
        approved_packages = response["ModelPackageSummaryList"]

        # Return error if no packages found
        if len(approved_packages) == 0:
            error_message = ("No approved ModelPackage found for ModelPackageGroup: {}".format(model_package_group_name))
            LOGGER.error("{}".format(error_message))

            raise Exception(error_message)

        model_package = approved_packages[0]
        LOGGER.info("Identified the latest approved model package: {}".format(model_package))

        return model_package
    except ClientError as e:
        stacktrace = traceback.format_exc()
        error_message = e.response["Error"]["Message"]
        LOGGER.error("{}".format(stacktrace))

        raise Exception(error_message)

In [ ]:
model_package_approved = get_approved_package(model_package_group_arn)
model_package = describe_model_package(model_package_approved["ModelPackageArn"])

In [ ]:
inference_component_name = "aws.samples.windturbine.detector"
inference_component_version = ""

model_component_name = "aws.samples.windturbine.model"
model_component_version = "{}.0.0".format(model_package["ModelPackageVersion"])

In [ ]:
# delete the ggv2 components
ggv2_client.delete_component(arn='arn:aws:greengrass:{}:{}:components:{}:versions:{}'.format(
    region,
    account_id,
    inference_component_name,
    inference_component_version
))

ggv2_client.delete_component(arn='arn:aws:greengrass:{}:{}:components:{}:versions:{}'.format(
    region,
    account_id,
    model_component_name,
    model_component_version
))

In [ ]:
# now deregister the devices from the fleet
resp = sm_client.list_devices(DeviceFleetName=fleet_name)
devices = [d['DeviceName'] for d in resp['DeviceSummaries']]
if len(devices) > 0:
    sm_client.deregister_devices(DeviceFleetName=fleet_name, DeviceNames=devices)

In [ ]:
# now deregister the devices from the fleet
for i,cert_arn in enumerate(iot_client.list_targets_for_policy(policyName=policy_name)['targets']):
    for t in iot_client.list_principal_things(principal=cert_arn)['things']:
        iot_client.detach_thing_principal(thingName=t, principal=cert_arn)
    iot_client.detach_policy(policyName=policy_name, target=cert_arn)
    certificateId = cert_arn.split('/')[-1]
    iot_client.update_certificate(certificateId=certificateId, newStatus='INACTIVE')
    iot_client.delete_certificate(certificateId=certificateId)

In [ ]:
iot_client.delete_role_alias(roleAlias='SageMakerEdge-%s' % fleet_name)

In [ ]:
s3_client.delete_object(
    Bucket=bucket_name,
    Key="wind_turbine_agent/config.tgz"
)

Mission Complete!  
Congratulations! :)